In [1]:
import pandas as pd
import numpy as np
from transformers import (
        XLMRobertaTokenizerFast,
    XLMRobertaForTokenClassification,
    get_scheduler
)

In [2]:
train_df=pd.read_csv('/kaggle/input/data-set1/indic-health-demo-main/Dataset/IHQID-WebMD/train.csv')[['question_hindi','disease_hindi','drug_hindi','treatment_hindi']]
test_df=pd.read_csv('/kaggle/input/data-set1/indic-health-demo-main/Dataset/IHQID-WebMD/test.csv')[['question_hindi','disease_hindi','drug_hindi','treatment_hindi']]

In [3]:
train_df.head()

,question_hindi,disease_hindi,drug_hindi,treatment_hindi
0,निस्टेटिन किस लिए निर्धारित किया गया है?,NaN,निस्टेटिन,NaN
1,क्या सम्भोग के बाद डाउचिंग मुझे गर्भवती होने स...,गर्भवती,NaN,NaN
2,क्या पेर्कोसेट वजन बढ़ने का कारण बनता है,भार बढ़ना,पर्कोसेट,NaN
3,क्या एक दिन में शराब के २ या २ १/२ गिलास उच्च ...,उच्च रक्तचाप,NaN,NaN
4,क्या बहुत अधिक छाछ थ्रश का कारण बन सकती है?,थ्रश,NaN,NaN


In [4]:
from deep_translator import GoogleTranslator
google_tranlate = GoogleTranslator(source='hi', target='en')


In [5]:
type(train_df['question_hindi'][0])

str

In [6]:
for i in range(len(train_df)):
    if type(train_df.loc[i,'question_hindi']) is not float:
        train_df.loc[i,'question_hindi']=google_tranlate.translate(train_df.loc[i,'question_hindi'])
    if type(train_df.loc[i,'disease_hindi']) is not float:
        train_df.loc[i,'disease_hindi']=google_tranlate.translate(train_df.loc[i,'disease_hindi'])
    if type(train_df.loc[i,'drug_hindi']) is not float:
        train_df.loc[i,'drug_hindi']=google_tranlate.translate(train_df.loc[i,'drug_hindi'])
    if type(train_df.loc[i,'treatment_hindi']) is not float:
        train_df.loc[i,'treatment_hindi']=google_tranlate.translate(train_df.loc[i,'treatment_hindi'])
for i in range(len(test_df)):
    if type(test_df.loc[i,'question_hindi']) is not float:
        test_df.loc[i,'question_hindi']=google_tranlate.translate(test_df.loc[i,'question_hindi'])
    if type(test_df.loc[i,'disease_hindi']) is not float:
        test_df.loc[i,'disease_hindi']=google_tranlate.translate(test_df.loc[i,'disease_hindi'])
    if type(test_df.loc[i,'drug_hindi']) is not float:
        test_df.loc[i,'drug_hindi']=google_tranlate.translate(test_df.loc[i,'drug_hindi'])
    if type(test_df.loc[i,'treatment_hindi']) is not float:
        test_df.loc[i,'treatment_hindi']=google_tranlate.translate(test_df.loc[i,'treatment_hindi'])
train_df=train_df.rename(columns={'question_hindi':'question_english','disease_hindi':'disease_english','drug_hindi':'drug_english','treatment_hindi':'treatment_english'})
test_df=test_df.rename(columns={'question_hindi':'question_english','disease_hindi':'disease_english','drug_hindi':'drug_english','treatment_hindi':'treatment_english'})


In [7]:
from fuzzywuzzy import fuzz

/opt/conda/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
def get_common_sequence(tokenized_sentence,tokenized_entity,entity,tag_list):
    common_sequence={
        'similarity':0.0,
        'start_index':-1,
        'end_index':-1
    }
    target_len=len(tokenized_entity)
    sentence_len=len(tokenized_sentence)
    for i in range(sentence_len-target_len):
        fuzz_ratio=fuzz.ratio(tokenized_sentence[i:i+target_len],tokenized_entity)
        if fuzz_ratio>=80 and common_sequence['similarity']<fuzz_ratio:
            common_sequence['similarity']=fuzz_ratio
            common_sequence['start_index']=i
            common_sequence['end_index']=i+target_len-1
        fuzz_ratio=fuzz.ratio(tokenized_sentence[i:i+target_len-1],tokenized_entity)
        if fuzz_ratio>=80 and common_sequence['similarity']<fuzz_ratio:
            common_sequence['similarity']=fuzz_ratio
            common_sequence['start_index']=i;
            common_sequence['end_index']=i+target_len-1
    tag_list[common_sequence['start_index']]="B-"+entity
    for i in range(common_sequence['start_index']+1,common_sequence['end_index']):
        tag_list[i]="I-"+entity

    

In [9]:
from nltk.tokenize import  word_tokenize
import math

In [10]:
for i in range(len(train_df)):
    tokenised_sentence=word_tokenize(train_df.loc[i,'question_english'].lower())
    tag_list=['O' for i in tokenised_sentence]
    if type(train_df.loc[i,'disease_english']) is not float:
        tokenized_disease=[word_tokenize(j.lower()) for j in train_df.loc[i,'disease_english'].split(',')]
        for k in tokenized_disease:
            get_common_sequence(tokenised_sentence,k,"disease",tag_list)
    else:
        assert(math.isnan(train_df.loc[i,'disease_english']))
    if type(train_df.loc[i,'drug_english']) is not float:
        tokenized_disease=[word_tokenize(j.lower()) for j in train_df.loc[i,'drug_english'].split(',')]
        for k in tokenized_disease:
            get_common_sequence(tokenised_sentence,k,"drug",tag_list)
    else:
        assert(math.isnan(train_df.loc[i,'drug_english']))
    if type(train_df.loc[i,'treatment_english']) is not float:
        tokenized_disease=[word_tokenize(j.lower()) for j in train_df.loc[i,'treatment_english'].split(',')]
        for k in tokenized_disease:
            get_common_sequence(tokenised_sentence,k,"treatment",tag_list)
    else:
        assert(math.isnan(train_df.loc[i,'treatment_english']))
    train_df.loc[i,'question_english']=str(tokenised_sentence)
    train_df.loc[i,'tag_english']=str(tag_list)

In [11]:
for i in range(len(test_df)):
    tokenised_sentence=word_tokenize(test_df.loc[i,'question_english'].lower())
    tag_list=['O' for i in tokenised_sentence]
    if type(test_df.loc[i,'disease_english']) is not float:
        tokenized_disease=[word_tokenize(j.lower()) for j in test_df.loc[i,'disease_english'].split(',')]
        for k in tokenized_disease:
            get_common_sequence(tokenised_sentence,k,"disease",tag_list)
    else:
        assert(math.isnan(test_df.loc[i,'disease_english']))
    if type(test_df.loc[i,'drug_english']) is not float:
        tokenized_disease=[word_tokenize(j.lower()) for j in test_df.loc[i,'drug_english'].split(',')]
        for k in tokenized_disease:
            get_common_sequence(tokenised_sentence,k,"drug",tag_list)
    else:
        assert(math.isnan(test_df.loc[i,'drug_english']))
    if type(test_df.loc[i,'treatment_english']) is not float:
        tokenized_disease=[word_tokenize(j.lower()) for j in test_df.loc[i,'treatment_english'].split(',')]
        for k in tokenized_disease:
            get_common_sequence(tokenised_sentence,k,"treatment",tag_list)
    else:
        assert(math.isnan(test_df.loc[i,'treatment_english']))
    test_df.loc[i,'question_english']=str(tokenised_sentence)
    test_df.loc[i,'tag_english']=str(tag_list)

In [12]:
train_df.head()

,question_english,disease_english,drug_english,treatment_english,tag_english
0,"['what', 'is', 'nystatin', 'prescribed', 'for'...",NaN,Nystatin,NaN,"['O', 'O', 'B-drug', 'O', 'O', 'O']"
1,"['can', 'douching', 'after', 'sex', 'prevent',...",pregnant,NaN,NaN,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-di..."
2,"['does', 'percocet', 'cause', 'weight', 'gain']",weight gain,Percocet,NaN,"['O', 'B-drug', 'O', 'O', 'B-disease']"
3,"['does', '2', 'or', '2', '1/2', 'glasses', 'of...",high blood pressure,NaN,NaN,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['can', 'too', 'much', 'buttermilk', 'cause', ...",Thrush,NaN,NaN,"['O', 'O', 'O', 'O', 'O', 'B-disease', 'O']"


In [13]:
def convert_string(string):
    special_remove=string.replace('[','').replace(']','').replace('"','').replace("'", '')
    split_str=special_remove.split(',')
    return [item.strip() for item in split_str]

In [14]:
train_df['question_english']=train_df['question_english'].apply(convert_string)
train_df['tag_english']=train_df['tag_english'].apply(convert_string)
test_df['question_english']=test_df['question_english'].apply(convert_string)
test_df['tag_english']=test_df['tag_english'].apply(convert_string)

In [15]:
label__ = {
    'O': 0,
    'B-treatment': 1,
    'I-treatment': 2,
    'B-disease': 3,
    'I-disease': 4,
    'B-drug': 5,
    'I-drug': 6
}

In [16]:
import torch
device='cuda' if torch.cuda.is_available() else 'cpu' 
device

'cuda'

In [17]:
model_checkpoint = "xlm-roberta-base"

hyper_parameters = {
    'batch_size': 8,
    'lr': 3e-5,
    'epochs': 10
}

In [18]:
def clean_list(lst):
    return [item for item in lst if item.strip()]
train_df['question_english']=train_df['question_english'].apply(clean_list)
test_df['question_english']=test_df['question_english'].apply(clean_list)

In [19]:
tokenizer=XLMRobertaTokenizerFast.from_pretrained(model_checkpoint,add_prefix_space=True)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
# def labelling(question,tag):
#     question = " ".join(question)
#     tokenized_input = tokenizer(question, truncation=True, padding='max_length', is_split_into_words=False)
#     word_ids = tokenized_input.word_ids()
#     for i, labels in enumerate(word_ids):
#         if labels is None:
#             word_ids[i] = 7
#         else:
#             word_ids[i] = label_1[tag[labels]]
#     tokenized_input["labels"] = word_ids
#     return tokenized_input
def process_queries(question, tag): 
    
    tokenized_input = tokenizer(question, max_length=300, truncation=True, padding='max_length', is_split_into_words=True)
    word_ids = tokenized_input.word_ids()
    j=1;
    for i, label in enumerate(word_ids):
        if label is None:
            word_ids[i] = 7
        else:
            word_ids[i] = label__[tag[label]]
            
    tokenized_input["labels"] = word_ids
    return tokenized_input

In [21]:
encoded_input_train = {
    'input_ids': [],
    'attention_mask': [],
    'tags_english': []
}
for index in range(len(train_df['question_english'])):
    process_output = process_queries(train_df.loc[index,'question_english'], train_df.loc[index,'tag_english'])
    encoded_input_train['input_ids'].append(process_output['input_ids'])
    encoded_input_train['attention_mask'].append(process_output['attention_mask'])
    encoded_input_train['tags_english'].append(process_output['labels'])

In [22]:
encoded_input_test = {
    'input_ids': [],
    'attention_mask': [],
    'tags_english': []
}
for index in range(len(test_df['question_english'])):
    process_output = process_queries(test_df.loc[index,'question_english'], test_df.loc[index,'tag_english'])
    encoded_input_test['input_ids'].append(process_output['input_ids'])
    encoded_input_test['attention_mask'].append(process_output['attention_mask'])
    encoded_input_test['tags_english'].append(process_output['labels'])

In [23]:
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm


In [24]:
train_dataloader = DataLoader(
    TensorDataset(
        torch.tensor(encoded_input_train['input_ids']).to(device),
        torch.tensor(encoded_input_train['attention_mask']).to(device),
        torch.tensor(encoded_input_train['tags_english']).to(device)
    ),
    batch_size=hyper_parameters['batch_size']
)
test_dataloader = DataLoader(
    TensorDataset(
        torch.tensor(encoded_input_test['input_ids']).to(device),
        torch.tensor(encoded_input_test['attention_mask']).to(device),
        torch.tensor(encoded_input_test['tags_english']).to(device)
    ),
    batch_size=hyper_parameters['batch_size']
)

In [25]:
model = XLMRobertaForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label__) + 1
)
model.to(device)

optimizer = AdamW(
    model.parameters(),
    lr=hyper_parameters['lr']
)

lr_scheduler = get_scheduler(
  "linear",
  optimizer=optimizer,
  num_warmup_steps=0,
  num_training_steps=hyper_parameters['epochs'] * len(train_dataloader)
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
model.train()

updater = tqdm(range(hyper_parameters['epochs']))
for epoch in updater:
    total_train_loss = 0.0
    for batch in train_dataloader:
        
        optimizer.zero_grad()
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2],
        }
        
        outputs = model(**inputs)
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        
        total_train_loss += loss.item()
    
    print("Epoch:", epoch + 1, " - Training Loss:", round(total_train_loss / len(train_dataloader), 4))


# Didn't have much time to implement early stopping. So, saving the model at the end of all epochs.
torch.save(model.state_dict(), f'ee_rob_en.model')

 10%|█         | 1/10 [00:36<05:27, 36.39s/it]

Epoch: 1  - Training Loss: 0.2105


 20%|██        | 2/10 [01:13<04:53, 36.68s/it]

Epoch: 2  - Training Loss: 0.0283


 30%|███       | 3/10 [01:52<04:23, 37.69s/it]

...

Epoch: 8  - Training Loss: 0.0088


 90%|█████████ | 9/10 [05:43<00:38, 38.54s/it]

Epoch: 9  - Training Loss: 0.0079


100%|██████████| 10/10 [06:22<00:00, 38.26s/it]

Epoch: 10  - Training Loss: 0.0074


In [28]:
model.eval()

prediction = []
gold_label = []

extra_appended_tokens = 0

for indexer, batch in enumerate(test_dataloader):

    inputs = {
        'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2],
    }

    with torch.no_grad():
        outputs = model(**inputs)


    gold_label_cpu = inputs['labels'].cpu().numpy()
    logits_vector = outputs.logits.detach().cpu().numpy()

    assert(len(gold_label_cpu) == len(logits_vector))

    for index in range(len(logits_vector)):
        prediction_vector = []
        for iterator__ in logits_vector[index].argmax(axis=1):
            if iterator__ != 7:
                prediction_vector.append(iterator__)
                prediction.append(iterator__)
        
        gold_label_vector = []
        for iterator__ in gold_label_cpu[index]:
            if iterator__ != 7:
                gold_label_vector.append(iterator__)
                gold_label.append(iterator__)
        
        # There are some cases (only observed once) when there was one mismatch in vector of gold label and prediction
        # To overcome that, for each tokenized
         while len(gold_label) < len(prediction):
            extra_appended_tokens += 1
            gold_label.append(0)
        
        while len(prediction) < len(gold_label):
            extra_appended_tokens += 1
            prediction.append(0)

print("Number of extra appended tokens : ", extra_appended_tokens)
print(classification_report(gold_label, prediction))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 40)